In [1]:
%pip install wikipedia
%pip install arxiv
%pip install langchainhub

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Tools

## Wikipedia

In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


In [3]:
wiki_api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)

wiki_tool = WikipediaQueryRun(api_wrapper=wiki_api_wrapper)

## Website

In [ ]:
# from langchain_community.document_loaders import WebBaseLoader
# from langchain_community.vectorstores import FAISS
# from langchain_ollama import OllamaEmbeddings
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# loader = WebBaseLoader(web_paths=["https://docs.smith.langchain.com/"]) #EXTRACT

# docs = loader.load() # LOAD

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs) # SPLIT

# db = FAISS.from_documents(text_splitter[:10], embedding=OllamaEmbeddings(model="llama3")) # EMBED

# retriever = db.as_retriever() # RETRIEVE



In [ ]:
# from langchain.tools.retriever import create_retriever_tool

# retriever_tool = create_retriever_tool(retriever, "langsmith_search", "Search for information about langsmith. For any question about langsmith, use this tool to answer.")


NameError: name 'retriever' is not defined

## Arxiv

In [4]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_api_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_api_wrapper)

In [5]:
tools = [wiki_tool, arxiv_tool]

# Agents

## LLM

In [6]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="C:/Users/Lenovo/Desktop/LANGCHAIN/.env", override=True)

True

In [7]:
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0)

In [9]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages  

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [14]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. You have access to a set of tools to answer user questions. Choose the best tool for the job."),
    ("user", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

## Agent

In [10]:
from langchain.agents import create_tool_calling_agent, AgentExecutor

agent = create_tool_calling_agent(llm, tools, prompt)

In [11]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [12]:
response2 = agent_executor.invoke({
    "input": "Who is the CEO of Tesla?"
})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Tesla CEO'}`
responded: [{'text': "To answer this question accurately, I'll need to look up the most current information about Tesla's leadership. Let me use the Wikipedia tool to find this information for you.", 'type': 'text', 'index': 0}, {'id': 'toolu_01LJZFAsRRysN2rMbvVWWVWS', 'input': {}, 'name': 'wikipedia', 'type': 'tool_use', 'index': 1, 'partial_json': '{"query": "Tesla CEO"}'}]

Page: Tesla, Inc.
Summary: Tesla, Inc. ( TEZ-lə or   TESS-lə) is an American multinational automotive and clean energy company. Headquartered in Austin, Texas, it designs, manufactures and sells batte
Invoking: `wikipedia` with `{'query': 'Elon Musk Tesla CEO'}`
responded: [{'text': "I apologize, but the summary provided doesn't contain the specific information we're looking for. Let me try a more direct search query to find the CEO of Tesla.", 'type': 'text', 'index': 0}, {'id': 'toolu_011yC122Z1nk18kkwzFd7rGg', 'input':

In [13]:
response2  

{'input': 'Who is the CEO of Tesla?',
 'output': [{'text': "I apologize for the difficulty in finding the specific information. Based on the search results and my knowledge, I can provide you with the answer:\n\nThe CEO of Tesla is Elon Musk.\n\nElon Musk has been the CEO of Tesla since 2008. He co-founded the company and has been a driving force behind its growth and innovation in the electric vehicle and clean energy sectors. While the Wikipedia tool didn't provide us with a direct statement about his current role, it's well-established that Elon Musk continues to serve as the CEO of Tesla.\n\nIf you need any more specific or up-to-date information about Tesla's leadership, please let me know, and I'll be happy to search for it.",
   'type': 'text',
   'index': 0}]}

In [14]:
response3 = agent_executor.invoke({
    "input": "Who are the authors of the 'BranchyNet: Fast Inference via Early Exiting from Deep Neural Networks' research paper?"
})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'BranchyNet: Fast Inference via Early Exiting from Deep Neural Networks'}`
responded: [{'text': "To answer this question, I'll need to search for information about this specific research paper. The arxiv tool would be most appropriate for this query since it's related to a scientific article in the field of Computer Science. Let me search for this paper.", 'type': 'text', 'index': 0}, {'id': 'toolu_01KgWHUvojbimdgeTaaWzi28', 'input': {}, 'name': 'arxiv', 'type': 'tool_use', 'index': 1, 'partial_json': '{"query": "BranchyNet: Fast Inference via Early Exiting from Deep Neural Networks"}'}]

Published: 2022-06-19
Title: Resource-Constrained Edge AI with Early Exit Prediction
Authors: Rongkang Dong, Yuyi Mao, Jun Zhang
Summary: By leveraging the data sample diversity, the early-exit networ
Invoking: `arxiv` with `{'query': '"BranchyNet: Fast Inference via Early Exiting from Deep Neural Networks" authors'}`
responded

In [17]:
response3["output"]

[{'text': 'Great! I\'ve found the correct paper. Now I can answer your question.\n\nThe authors of the research paper "BranchyNet: Fast Inference via Early Exiting from Deep Neural Networks" are:\n\n1. Surat Teerapittayanon\n2. Bradley McDanel\n3. H. T. Kung\n\nThis paper was published on September 6, 2017, according to the arxiv search results.',
  'type': 'text',
  'index': 0}]

In [18]:
response4 = agent_executor.invoke({
    "input": "How many research papers are based on the BranchyNet architecture?"
})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'BranchyNet'}`
responded: [{'text': "To answer this question accurately, we'll need to search for scientific articles related to the BranchyNet architecture. The best tool for this purpose is the arxiv function, which allows us to search for research papers in various scientific fields, including Computer Science.\n\nLet's use the arxiv tool to search for papers mentioning BranchyNet:", 'type': 'text', 'index': 0}, {'id': 'toolu_016NV4AgygdYTctURa7Z9Aqu', 'input': {}, 'name': 'arxiv', 'type': 'tool_use', 'index': 1, 'partial_json': '{"query": "BranchyNet"}'}]

Published: 2017-09-06
Title: BranchyNet: Fast Inference via Early Exiting from Deep Neural Networks
Authors: Surat Teerapittayanon, Bradley McDanel, H. T. Kung
Summary: Deep neural networks are state 
Invoking: `arxiv` with `{'query': '"BranchyNet" OR "Branchy Net"'}`
responded: [{'text': 'Based on the search results, we can see that there is at least one 